In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import os
os.chdir('/content/drive/My Drive/Pose Estimation/orvee/Pretrained openpose')

Mounted at /content/drive


In [ ]:
!pip install tensorflow==2.2

     |████████████████████████████████| 516.2MB 29kB/s 
     |████████████████████████████████| 3.0MB 45.8MB/s 
     |████████████████████████████████| 460kB 37.4MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
from __future__ import division
import os
import cv2
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import os
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import Model, Sequential, utils
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import load_model

import pandas as pd 
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
from random import shuffle
import os
from keras.models import Model, Sequential, load_model
from keras import layers
from keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D, ZeroPadding2D, concatenate, Input, Permute, Concatenate, BatchNormalization
from keras import applications
#from keras.utils import plot_model, to_categorical
from keras.applications import VGG16, ResNet50
from keras.optimizers import RMSprop,Adam  
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
import pickle
import random
import cv2

In [ ]:
# DATA LOADER
data_dict = dict()
data_dict_test = dict()
validation_dict = dict()
test_dict = dict()

data_dict['X'] = np.load('bangla_dataset/np_files/imglist_t.npy', allow_pickle=True) #DATASET.train_folder + '/images.npy'
data_dict['X2'] = np.load('bangla_dataset/np_files/poselist_t.npy', allow_pickle=True)
data_dict['Y'] = np.load('bangla_dataset/new/labellist_t.npy', allow_pickle=True)

data_dict_test['X'] = np.load('bangla_dataset/new/imglist_v.npy', allow_pickle=True) #DATASET.train_folder + '/images.npy'
data_dict_test['X2'] = np.load('bangla_dataset/new/poselist_v.npy', allow_pickle=True)
data_dict_test['Y'] = np.load('bangla_dataset/new/labellist_v.npy', allow_pickle=True)

#FINAL64_ONLY_CNN

In [ ]:
image_network = Input(shape = [64, 64, 1], name = "input_1")
x=Convolution2D(64, (3, 3), padding = 'same', input_shape = (64, 64))(image_network)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=Convolution2D(64, (3, 3), padding = 'same')(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=MaxPooling2D(pool_size=(2, 2))(x)
x=Dropout(0.2)(x)

x=Convolution2D(128, (3, 3), padding = 'same')(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=Convolution2D(128, (3, 3), padding = 'same')(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=MaxPooling2D(pool_size=(2, 2))(x)
x=Dropout(0.2)(x)

x=Convolution2D(256, (3, 3), padding = 'same', input_shape = (64, 64))(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=Convolution2D(256, (3, 3), padding = 'same', input_shape = (64, 64))(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=Convolution2D(256, (3, 3), padding = 'same', input_shape = (64, 64))(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=MaxPooling2D(pool_size=(2, 2))(x)
x=Dropout(0.2)(x)

x=Convolution2D(512, (3, 3), padding = 'same', input_shape = (64, 64))(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=Convolution2D(512, (3, 3), padding = 'same', input_shape = (64, 64))(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=Convolution2D(512, (3, 3), padding = 'same', input_shape = (64, 64))(x)
x=Activation('relu')(x)
x=BatchNormalization()(x)
x=MaxPooling2D(pool_size=(2, 2))(x)
x=Dropout(0.3)(x)

x = Flatten()(x)
x = Dense(1024, activation = 'relu')(x)
x=BatchNormalization()(x)
x=Dropout(0.3)(x)
w = Dense(512, activation = 'relu')(x)
w=Dropout(0.3)(w)
# pose_network = Input(shape = [21, 2], name = "input_2")

# x2 = Flatten()(pose_network)
# x2 = Dense(1024, activation = 'relu')(x2)
# x2=Dropout(0.3)(x2)
# z = Dense(512, activation = 'relu')(x2)
# z=Dropout(0.3)(z)


# merged_network = concatenate([w, z], axis = 1)
# merged_network = Dense(512, activation = 'elu')(merged_network)
# merged_network = Dense(256, activation = 'elu')(merged_network)
# merged_network=BatchNormalization()(merged_network)
merged_network = Dense(38, activation = 'softmax')(w)

model = Model(inputs = image_network, outputs = merged_network)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 64, 64)        640       
_________________________________________________________________
activation (Activation)      (None, 64, 64, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        36928     
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 64)        256   

In [ ]:
model = load_model("final_weights/final64_only_cnn_epoch25th.h5")

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=.000001),
             metrics=["accuracy"])

In [ ]:
checkpoint = ModelCheckpoint(
                             'final_weights/final64_only_cnn_epoch30.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True, 
                             verbose=1)

earlystop = EarlyStopping(                   
                          monitor='val_loss',
                          min_delta=0,
                          restore_best_weights=True,
                          patience=20,
                          verbose=1)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.000001)

callbacks = [earlystop,checkpoint,learning_rate_reduction]

In [ ]:
H = model.fit(data_dict['X'],
	data_dict['Y'],
	validation_data=(data_dict_test['X'],
	data_dict_test['Y']),
	epochs=25,
  callbacks=callbacks,
	verbose=1)

Epoch 1/25
312/312 [==============================] - ETA: 0s - loss: 3.3205 - accuracy: 0.2019
Epoch 00001: val_loss improved from inf to 3.74463, saving model to final_weights/final64_only_cnn_epoch20.h5
312/312 [==============================] - 1632s 5s/step - loss: 3.3205 - accuracy: 0.2019 - val_loss: 3.7446 - val_accuracy: 0.2305 - lr: 0.0010
Epoch 2/25
312/312 [==============================] - ETA: 0s - loss: 1.7825 - accuracy: 0.4860
Epoch 00002: val_loss improved from 3.74463 to 3.25563, saving model to final_weights/final64_only_cnn_epoch20.h5
312/312 [==============================] - 1652s 5s/step - loss: 1.7825 - accuracy: 0.4860 - val_loss: 3.2556 - val_accuracy: 0.1343 - lr: 0.0010
Epoch 3/25
312/312 [==============================] - ETA: 0s - loss: 1.3695 - accuracy: 0.5840
Epoch 00003: val_loss improved from 3.25563 to 1.24411, saving model to final_weights/final64_only_cnn_epoch20.h5
312/312 [==============================] - 1669s 5s/step - loss: 1.3695 - accuracy

In [ ]:
H = model.fit(data_dict['X'],
	data_dict['Y'],
	validation_data=(data_dict_test['X'],
	data_dict_test['Y']),
	epochs=5,
  callbacks=callbacks,
	verbose=1)

Epoch 1/5
312/312 [==============================] - ETA: 0s - loss: 0.0632 - accuracy: 0.9795
Epoch 00001: val_loss improved from inf to 0.22073, saving model to final_weights/final64_only_cnn_epoch30.h5
312/312 [==============================] - 1637s 5s/step - loss: 0.0632 - accuracy: 0.9795 - val_loss: 0.2207 - val_accuracy: 0.9356 - lr: 1.0000e-06
Epoch 2/5
312/312 [==============================] - ETA: 0s - loss: 0.0658 - accuracy: 0.9778
Epoch 00002: val_loss improved from 0.22073 to 0.21737, saving model to final_weights/final64_only_cnn_epoch30.h5
312/312 [==============================] - 1636s 5s/step - loss: 0.0658 - accuracy: 0.9778 - val_loss: 0.2174 - val_accuracy: 0.9347 - lr: 1.0000e-06
Epoch 3/5
312/312 [==============================] - ETA: 0s - loss: 0.0661 - accuracy: 0.9804
Epoch 00003: val_loss did not improve from 0.21737
312/312 [==============================] - 1632s 5s/step - loss: 0.0661 - accuracy: 0.9804 - val_loss: 0.2177 - val_accuracy: 0.9338 - lr: 1

In [ ]:
model = model.save("final_weights/final64_only_cnn_epoch30th.h5")

In [ ]:
model = load_model("final_weights/final64_only_cnn_epoch20.h5")

In [ ]:
model.save("trained_weights/vgg16_new_elu_epoch25.h5")

NameError: ignored

In [ ]:
data_dict_evaluate = dict()
data_dict_evaluate['X'] = np.load('imglist_test.npy', allow_pickle=True)
data_dict_evaluate['X2'] = np.load('poselist_test.npy', allow_pickle=True)
data_dict_evaluate['Y'] = np.load('labellist_test.npy', allow_pickle=True)


In [ ]:
model = load_model("final_weights/final64_only_cnn_epoch20.h5")

In [ ]:
score = model.evaluate({"ip_img": data_dict_evaluate['X'], "ip_pose": data_dict_evaluate['X2']},
	data_dict_evaluate['Y'])

In [ ]:
score = model.evaluate(data_dict_evaluate['X'],
	data_dict_evaluate['Y'])

48/48 [==============================] - 59s 1s/step - loss: 1.1880 - accuracy: 0.6395


In [ ]:
model = load_model('final_weights/final64_only_cnn_epoch30th.h5')

In [ ]:
score = model.evaluate(data_dict_evaluate['X'],
	data_dict_evaluate['Y'])

48/48 [==============================] - 60s 1s/step - loss: 0.4010 - accuracy: 0.9000


In [ ]:
score = model.evaluate(data_dict_evaluate['X'],
	data_dict_evaluate['Y'])

48/48 [==============================] - 58s 1s/step - loss: 0.4010 - accuracy: 0.9000
